In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D,Dropout, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
!git clone https://github.com/RishitToteja/Chext-X-ray-Images-Data-Set.git
# used data set from github source

Cloning into 'Chext-X-ray-Images-Data-Set'...
remote: Enumerating objects: 2319, done.
remote: Total 2319 (delta 0), reused 0 (delta 0), pack-reused 2319
Receiving objects: 100% (2319/2319), 1.31 GiB | 36.02 MiB/s, done.
Checking out files: 100% (2296/2296), done.


In [ ]:
import os

main_dir = "/content/Chext-X-ray-Images-Data-Set/DataSet/Data"

# SETTING TRAIN AND TEST DIRECTORY
train_dir = os.path.join(main_dir, "train")
test_dir = os.path.join(main_dir, "test")

#SETING DIRECTORY FOR COVID AND NORMAL IMAGES DIRECTORY
train_covid_dir = os.path.join(train_dir, "COVID19")
train_normal_dir = os.path.join(train_dir, "NORMAL")

test_covid_dir = os.path.join(test_dir, "COVID19")
test_normal_dir = os.path.join(test_dir, "NORMAL")
train_covid_names = os.listdir(train_covid_dir)
train_normal_names = os.listdir(train_normal_dir)

test_covid_names = os.listdir(test_covid_dir)
test_normal_names = os.listdir(test_normal_dir)

In [ ]:
dgen_train = ImageDataGenerator(rescale = 1./255,
                                validation_split = 0.2,
                                zoom_range = 0.2,
                                horizontal_flip = True)

dgen_validation = ImageDataGenerator(rescale = 1./255,
                                     )

dgen_test = ImageDataGenerator(rescale = 1./255,
                              )

train_generator = dgen_train.flow_from_directory(train_dir,
                                                 target_size = (150, 150), 
                                                 subset = 'training',
                                                 batch_size = 32,
                                                 class_mode = 'binary')
validation_generator = dgen_train.flow_from_directory(train_dir,
                                                      target_size = (150, 150), 
                                                      subset = "validation", 
                                                      batch_size = 32, 
                                                      class_mode = "binary")
test_generator = dgen_test.flow_from_directory(test_dir,
                                               target_size = (150, 150), 
                                               batch_size = 32, 
                                               class_mode = "binary")

Found 1449 images belonging to 2 classes.
Found 362 images belonging to 2 classes.
Found 484 images belonging to 2 classes.


In [ ]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D

model = Sequential()

# 1) CONVOLUTIONAL LAYER - 1
model.add(Conv2D(32, (5,5), padding = "same", activation = "relu", input_shape = train_generator.image_shape))

# 2) POOLING LAYER - 1
model.add(MaxPooling2D(pool_size=(2,2)))

# 3) DROPOUT LAYER -2
model.add(Dropout(0.5))

# 4) CONVOLUTIONAL LAYER - 2
model.add(Conv2D(64, (5,5), padding = "same", activation = "relu"))

# 5) POOLING LAYER - 2
model.add(MaxPooling2D(pool_size=(2,2)))

# 6) DROPOUT LAYER - 2
model.add(Dropout(0.5))

# 7) FLATTENING LAYER TO 2D SHAPE
model.add(Flatten())

# 8) ADDING A DENSE LAYER
model.add(Dense(256, activation = 'relu'))

# 9 DROPOUT LAYER - 3
model.add(Dropout(0.5))

# 10) FINAL OUTPUT LAYER
model.add(Dense(1, activation = 'sigmoid'))

In [ ]:
model.compile(Adam(learning_rate = 0.001), loss = 'binary_crossentropy', metrics = ['accuracy'])

In [ ]:
history = model.fit(train_generator, 
                    epochs = 35, 
                    validation_data = validation_generator)

Epoch 1/35
46/46 [==============================] - 165s 4s/step - loss: 1.0485 - accuracy: 0.7081 - val_loss: 0.3087 - val_accuracy: 0.8785
Epoch 2/35
46/46 [==============================] - 164s 4s/step - loss: 0.2430 - accuracy: 0.9027 - val_loss: 0.1757 - val_accuracy: 0.9586
Epoch 3/35
46/46 [==============================] - 161s 3s/step - loss: 0.1779 - accuracy: 0.9427 - val_loss: 0.1483 - val_accuracy: 0.9530
Epoch 4/35
46/46 [==============================] - 162s 4s/step - loss: 0.1615 - accuracy: 0.9476 - val_loss: 0.1494 - val_accuracy: 0.9558
Epoch 5/35
46/46 [==============================] - 164s 4s/step - loss: 0.1838 - accuracy: 0.9379 - val_loss: 0.1454 - val_accuracy: 0.9530
Epoch 6/35
46/46 [==============================] - 161s 3s/step - loss: 0.1506 - accuracy: 0.9441 - val_loss: 0.1643 - val_accuracy: 0.9586
Epoch 7/35
46/46 [==============================] - 160s 3s/step - loss: 0.1304 - accuracy: 0.9572 - val_loss: 0.1346 - val_accuracy: 0.9641
Epoch 8/35
46

In [ ]:
model.save("model.h5")

In [ ]:
from google.colab import files
from keras.preprocessing import image
from tensorflow.keras.utils import load_img, img_to_array 

uploaded = files.upload()

for filename in uploaded.keys():
  img_path = '/content/' + filename
  img = load_img(img_path, target_size = (150,150))
  images = img_to_array(img)
  images = np.expand_dims(images, axis = 0)
  prediction = model.predict(images)
  
  if prediction == 0:
    print("The report is COVID-19 Positive")
  else:
    print("The report is COVID-19 Negative")

Saving COVID-19 (313).jpg to COVID-19 (313) (3).jpg
1/1 [==============================] - 0s 190ms/step
The report is COVID-19 Positive
